## Decorators and Closures

In [1]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'Function {func.__name__} took {end - start} seconds to execute')
        return result
    return wrapper

@time_it
def some_function():
    # Your code here
    return "Done"

some_function()

Function some_function took 9.5367431640625e-07 seconds to execute


'Done'

In [2]:
## Calling some_function like above will be the same as the following code
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'Function {func.__name__} took {end - start} seconds to execute')
        return result
    return wrapper

def some_function():
    # Your code here
    return "Done"

time_it(some_function)()

Function some_function took 9.5367431640625e-07 seconds to execute


'Done'

Three essential facts make a good summary of decorators:
- A decorator is a function or another callable.

- A decorator may replace the decorated function with a different one.

- Decorators are executed immediately when a module is loaded.

## Closures

In [3]:
class Averager:

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [4]:
avg = Averager()

In [5]:
avg

In [6]:
avg(10)

10.0

In [7]:
avg(11)

10.5

### Functional Implementation, using higher-order function

In [8]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

In [9]:
avg = make_averager()

In [10]:
avg(10)

10.0

In [11]:
avg(12)

11.0

It’s obvious where the `avg` of the Averager class keeps the history: the `self.series` instance attribute. But where does the `avg` function in the second example find the series?

Note that `series` is a local variable of `make_averager` because the assignment `series = []` happens in the body of that function. But when `avg(10)` is called, `make_averager` has already returned, and its local scope is long gone.

Within `averager`, `series` is a free variable. This is a technical term meaning a variable that is not bound in the local scope

Inspecting the returned `averager` object shows how Python keeps the names of local and free variables in the `__code__` attribute that represents the compiled body of the function

In [12]:
avg.__code__.co_freevars

('series',)

In [13]:
avg.__closure__

(<cell at 0x105d9cd00: list object at 0x105d4d4c0>,)

In [14]:
avg.__closure__[0].cell_contents

[10, 12]

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.

## The nonlocal Declaration

Our previous implementation of `make_averager` was not efficient. We stored all the values in the historical series and computed their sum every time averager was called. A better implementation would only store the total and the number of items so far, and compute the mean from these two numbers.

`nonlocal` lets you declare a variable as a free variable even when it is assigned within the function. If a new value is assigned to a `nonlocal` variable, the binding stored in the closure is changed.

In [15]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1  # without nonlocal, this wouldn't work because count+=1 will
        total += new_value # reassign the count variable with the averager scope
        return total / count

    return averager

### Variable Lookup Logic

When a function is defined, the Python bytecode compiler determines how to fetch a variable x that appears in it, based on these rules:

- If there is a global x declaration, x comes from and is assigned to the x global variable module.
- If there is a nonlocal x declaration, x comes from and is assigned to the x local variable of the nearest surrounding function where x is defined.
- If x is a parameter or is assigned a value in the function body, then x is the local variable.
- If x is referenced but is not assigned and is not a parameter:
    - x will be looked up in the local scopes of the surrounding function bodies (nonlocal scopes).
    - If not found in surrounding scopes, it will be read from the module global scope.
    - If not found in the global scope, it will be read from `__builtins__.__dict__.`

Now that we have Python closures covered, we can effectively implement decorators with nested functions.

## Parameterized Decorators

In [16]:
registry = set()

def register(active=False):
    def decorate(func):
        print('running register'
              f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)

        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x105db1ea0>)
running register(active=False)->decorate(<function f2 at 0x105db1fc0>)


In [17]:
f1()

running f1()


In [18]:
register(f2())

running f2()


<function __main__.register.<locals>.decorate(func)>

f1 and f2 functions when being called will be passed to the `decorate` function inside `register` function instead because `register` function already have default parameter implemented. Hence, the f1 and f2 will be passed to the next available inner function inside `register`.